In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

In [66]:
import pandas as pd
df = pd.read_csv('insurance.csv')
print(df.isnull().sum())

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64


In [68]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1193 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1193 non-null   int64  
 1   sex       1193 non-null   object 
 2   bmi       1193 non-null   float64
 3   children  1193 non-null   int64  
 4   smoker    1193 non-null   object 
 5   region    1193 non-null   object 
 6   charges   1193 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 74.6+ KB


In [16]:
numeric_features = df.select_dtypes(include=['number']).columns
initial_shape = df.shape

# Расчет IQR только для числовых признаков
Q1 = df[numeric_features].quantile(0.25)
Q3 = df[numeric_features].quantile(0.75)
IQR = Q3 - Q1

# Создание маски для фильтрации выбросов
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Применение фильтрации
mask = ~((df[numeric_features] < lower_bound) | (df[numeric_features] > upper_bound)).any(axis=1)
df_clean = df[mask].copy()

print(f"Удалено строк с выбросами: {initial_shape[0] - df_clean.shape[0]}")
print(f"Осталось строк после удаления выбросов: {df_clean.shape[0]}")

Удалено строк с выбросами: 62
Осталось строк после удаления выбросов: 1131


In [18]:
categorical_features = df_clean.select_dtypes(exclude=['number']).columns.tolist()
print(categorical_features)

X = df_clean.drop('charges', axis=1)
y = df_clean['charges']

encoder = OneHotEncoder(drop='first', sparse_output=False)
cat_features = X[categorical_features]
encoded_features = encoder.fit_transform(cat_features)

# Создание датафрейма с закодированными признаками
encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_features))
encoded_df.index = X.index

# Объединение с числовыми признаками
numeric_features = X.select_dtypes(include=['number']).columns.tolist()
X_encoded = pd.concat([X[numeric_features], encoded_df], axis=1)

# Добавление столбца единиц для bias-члена
X_encoded = pd.concat([pd.Series(1, index=X_encoded.index, name='bias'), X_encoded], axis=1)

['sex', 'smoker', 'region']


In [19]:
correlation_matrix = pd.concat([X_encoded.drop('bias', axis=1), y], axis=1).corr()
correlation_matrix

,age,bmi,children,sex_male,smoker_yes,region_northwest,region_southeast,region_southwest,charges
age,1.000000,0.137661,0.034258,-0.014722,-0.085327,-0.009405,-0.022787,0.030540,0.506768
bmi,0.137661,1.000000,-0.016373,0.012600,-0.270385,-0.111903,0.232258,0.012655,-0.076329
children,0.034258,-0.016373,1.000000,0.009049,0.033819,0.013057,-0.021590,0.026780,0.129033
sex_male,-0.014722,0.012600,0.009049,1.000000,0.017368,0.008776,-0.003674,0.000524,-0.028048
smoker_yes,-0.085327,-0.270385,0.033819,0.017368,1.000000,0.003679,-0.007498,-0.029565,0.588291
region_northwest,-0.009405,-0.111903,0.013057,0.008776,0.003679,1.000000,-0.333719,-0.335299,-0.000951
region_southeast,-0.022787,0.232258,-0.021590,-0.003674,-0.007498,-0.333719,1.000000,-0.332154,-0.040207
region_southwest,0.030540,0.012655,0.026780,0.000524,-0.029565,-0.335299,-0.332154,1.000000,-0.041037
charges,0.506768,-0.076329,0.129033,-0.028048,0.588291,-0.000951,-0.040207,-0.041037,1.000000


In [20]:
target_correlations = correlation_matrix['charges'].sort_values(ascending=False)
print(target_correlations)

charges             1.000000
smoker_yes          0.588291
age                 0.506768
children            0.129033
region_northwest   -0.000951
sex_male           -0.028048
region_southeast   -0.040207
region_southwest   -0.041037
bmi                -0.076329
Name: charges, dtype: float64


In [21]:
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded.values, y.values, test_size=0.2, random_state=42
)

In [22]:
# Стандартизация данных для градиентного спуска
scaler = StandardScaler()
# Стандартизируем только не-bias признаки (начиная со второго столбца)
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

X_train_scaled[:, 1:] = scaler.fit_transform(X_train[:, 1:])
X_test_scaled[:, 1:] = scaler.transform(X_test[:, 1:])

In [23]:
print("\nВычисление аналитического решения линейной регрессии...")
X_train_no_bias = X_train[:, 1:]  # Убираем bias для сравнения с sklearn
X_train_with_bias = X_train  # С bias для нашей реализации

# Формула: w = (X^T X)^-1 X^T y
XTX = X_train_with_bias.T @ X_train_with_bias
XTy = X_train_with_bias.T @ y_train

# Добавляем небольшое значение для численной устойчивости
XTX_inv = np.linalg.inv(XTX + 1e-8 * np.eye(XTX.shape[0]))
w_analytic = XTX_inv @ XTy


Вычисление аналитического решения линейной регрессии...


In [24]:
print("Аналитическое решение (веса):")
print(w_analytic)

Аналитическое решение (веса):
[-2163.95667363   235.52312962    41.92339216   483.1810894
  -409.39598618 12952.30772929  -665.69011825 -1272.92685706
 -1317.88512377]


In [ ]:
def gradient_descent(X, y, learning_rate=0.01, n_iterations=1000, tol=1e-6):
    """
    Реализация градиентного спуска для линейной регрессии
    
    Параметры:
    X - матрица признаков с bias
    y - вектор целевых значений
    learning_rate - скорость обучения
    n_iterations - количество итераций
    tol - порог для остановки по изменению функции потерь
    
    Возвращает:
    w - оптимальные веса
    loss_history - история значений функции потерь
    """
    n_samples, n_features = X.shape
    w = np.random.normal(0, 1, n_features)
    loss_history = []
    
    for i in range(n_iterations):
        # Предсказания
        y_pred = X @ w
        
        # Функция потерь (MSE)
        loss = np.mean((y_pred - y) ** 2)
        loss_history.append(loss)
        
        # Градиент функции потерь
        gradient = (2 / n_samples) * X.T @ (y_pred - y)
        
        # Обновление весов
        w_new = w - learning_rate * gradient
        
        # Проверка сходимости
        if np.linalg.norm(w_new - w) < tol:
            print(f"Градиентный спуск сошелся на итерации {i}")
            w = w_new
            break
            
        w = w_new
        
        # Вывод прогресса каждые 100 итераций
        if i % 100 == 0:
            print(f"Итерация {i}, Loss: {loss:.4f}")
    
    return w, loss_history

In [48]:
w_gd, loss_history = gradient_descent(X_train_scaled, y_train, learning_rate=0.1, n_iterations=5000)

Итерация 0, Loss: 116719916.8468
Итерация 100, Loss: 10637918.1946
Итерация 200, Loss: 10637917.6897
Градиентный спуск сошелся на итерации 274


In [32]:
w_gd

array([9058.24223918, 3253.30674448,  244.04863945,  590.400701  ,
       -204.40923391, 3839.4157235 , -289.51648378, -554.4072218 ,
       -568.96766362])

In [34]:
# Аналитическое решение с регуляризацией (Ridge)
def ridge_regression_analytic(X, y, alpha=1.0):
    """
    Аналитическое решение Ridge регрессии
    
    Параметры:
    X - матрица признаков с bias
    y - вектор целевых значений
    alpha - коэффициент регуляризации
    
    Возвращает:
    w - оптимальные веса с регуляризацией
    """
    n_features = X.shape[1]
    # Матрица регуляризации (кроме bias)
    reg_matrix = alpha * np.eye(n_features)
    reg_matrix[0, 0] = 0  # Не регуляризуем bias
    
    XTX = X.T @ X
    XTy = X.T @ y
    
    # Решение с регуляризацией
    w = np.linalg.inv(XTX + reg_matrix) @ XTy
    return w

In [38]:
# Численное решение с регуляризацией
def ridge_regression_gd(X, y, alpha=1.0, learning_rate=0.01, n_iterations=1000, tol=1e-6):
    """
    Градиентный спуск для Ridge регрессии
    
    Параметры:
    X - матрица признаков с bias
    y - вектор целевых значений
    alpha - коэффициент регуляризации
    learning_rate - скорость обучения
    n_iterations - количество итераций
    
    Возвращает:
    w - оптимальные веса
    loss_history - история значений функции потерь
    """
    n_samples, n_features = X.shape
    w = np.zeros(n_features)
    loss_history = []
    
    for i in range(n_iterations):
        # Предсказания
        y_pred = X @ w
        
        # Функция потерь (MSE + регуляризация)
        mse_loss = np.mean((y_pred - y) ** 2)
        reg_loss = alpha * np.sum(w[1:] ** 2)  # Регуляризация без bias
        loss = mse_loss + reg_loss
        loss_history.append(loss)
        
        # Градиент функции потерь с регуляризацией
        gradient = (2 / n_samples) * X.T @ (y_pred - y + alpha)
        # Добавляем градиент регуляризации (кроме bias)
        reg_gradient = 2 * alpha * w
        reg_gradient[0] = 0  # Не регуляризуем bias
        gradient += reg_gradient
        
        # Обновление весов
        w_new = w - learning_rate * gradient
        
        # Проверка сходимости
        if np.linalg.norm(w_new - w) < tol:
            print(f"Ridge градиентный спуск сошелся на итерации {i}")
            w = w_new
            break
            
        w = w_new
        
        # Вывод прогресса каждые 100 итераций
        if i % 100 == 0:
            print(f"Ridge итерация {i}, Loss: {loss:.4f}")
    
    return w, loss_history

In [78]:
# Подбор оптимального параметра регуляризации
alphas = [0.01, 0.1, 1.0, 10.0, 100.0]
print("\nПодбор оптимального параметра регуляризации...")
best_alpha_anal = None
best_mse = float('inf')
mse_results = []

for alpha in alphas:
    # Аналитическое решение с регуляризацией
    w_ridge = ridge_regression_analytic(X_train_with_bias, y_train, alpha)
    
    # Предсказания на тестовой выборке
    y_pred = X_test @ w_ridge
    mse = mean_squared_error(y_test, y_pred)
    mse_results.append(mse)
    
    print(f"Alpha = {alpha:.2f}, MSE на тесте = {mse:.2f}")
    
    if mse < best_mse:
        best_mse = mse
        best_alpha_anal = alpha


Подбор оптимального параметра регуляризации...
Alpha = 0.01, MSE на тесте = 14783552.35
Alpha = 0.10, MSE на тесте = 14781252.91
Alpha = 1.00, MSE на тесте = 14760720.66
Alpha = 10.00, MSE на тесте = 14748345.11
Alpha = 100.00, MSE на тесте = 18077228.74


In [80]:
# Подбор оптимального параметра регуляризации
alphas = [0.01, 0.1, 1.0, 10.0, 100.0]
print("\nПодбор оптимального параметра регуляризации...")
best_alpha_gd = None
best_mse = float('inf')
mse_results = []
weights_history = {}

for alpha in alphas:
    print(f"\nОбучение с alpha = {alpha:.2f}")
    
    # Используем стандартизованные данные для градиентного спуска
    w_ridge, loss_history = ridge_regression_gd(
        X_train_scaled, y_train, alpha=alpha, 
        learning_rate=0.01, n_iterations=2000
    )
    
    # Проверка на наличие NaN в весах
    if np.isnan(w_ridge).any():
        print(f"ВНИМАНИЕ: веса содержат NaN при alpha={alpha}. Пропускаем этот параметр.")
        mse = float('inf')
        mse_results.append(mse)
        continue
    
    weights_history[alpha] = w_ridge
    
    # Для предсказания используем функцию predict с правильной стандартизацией
    y_pred = predict(X_test, w_ridge, scaler=scaler, has_bias=True)
    
    # Дополнительная проверка на NaN в предсказаниях
    if np.isnan(y_pred).any():
        print(f"ВНИМАНИЕ: предсказания содержат NaN при alpha={alpha}. Заменяем на константу.")
        y_pred = np.full_like(y_pred, np.mean(y_train))
    
    mse = mean_squared_error(y_test, y_pred)
    mse_results.append(mse)
    
    print(f"Alpha = {alpha:.2f}, MSE на тесте = {mse:.2f}, Последний Loss = {loss_history[-1]:.4f}")
    
    if mse < best_mse:
        best_mse = mse
        best_alpha_gd = alpha

print(f"\nОптимальный alpha = {best_alpha_gd:.2f} с MSE = {best_mse:.2f}")


Подбор оптимального параметра регуляризации...

Обучение с alpha = 0.01
Ridge итерация 0, Loss: 116719916.8468
Ridge итерация 100, Loss: 12987429.1241
Ridge итерация 200, Loss: 10960204.2286
Ridge итерация 300, Loss: 10905376.2440
Ridge итерация 400, Loss: 10901008.3000
Ridge итерация 500, Loss: 10900095.2105
Ridge итерация 600, Loss: 10899856.9006
Ridge итерация 700, Loss: 10899792.6838
Ridge итерация 800, Loss: 10899775.2890
Ridge итерация 900, Loss: 10899770.5725
Ridge итерация 1000, Loss: 10899769.2933
Ridge итерация 1100, Loss: 10899768.9464
Ridge итерация 1200, Loss: 10899768.8523
Ridge итерация 1300, Loss: 10899768.8268
Ridge итерация 1400, Loss: 10899768.8199
Ridge итерация 1500, Loss: 10899768.8180
Ridge итерация 1600, Loss: 10899768.8175
Ridge итерация 1700, Loss: 10899768.8174
Ridge итерация 1800, Loss: 10899768.8173
Ridge итерация 1900, Loss: 10899768.8173
Alpha = 0.01, MSE на тесте = 14759774.49, Последний Loss = 10899768.8173

Обучение с alpha = 0.10
Ridge итерация 0, Lo

In [83]:
# Аналитическое решение с оптимальным alpha
w_ridge_analytic = ridge_regression_analytic(X_train_with_bias, y_train, best_alpha_anal)
print("Аналитическое решение Ridge (веса):")
print(w_ridge_analytic)

# Численное решение с оптимальным alpha
w_ridge_gd, ridge_loss_history = ridge_regression_gd(
    X_train_scaled, y_train, alpha=best_alpha_gd, learning_rate=0.05, n_iterations=5000
)
print("Численное решение Ridge (веса):")
print(w_ridge_gd)

Аналитическое решение Ridge (веса):
[-1416.31793109   233.89709646    18.76658584   489.83035336
  -370.06231875 11398.04697791  -528.78141115 -1050.6743413
 -1151.01968782]
Ridge итерация 0, Loss: 116719916.8468
Ridge итерация 100, Loss: 10900067.4791
Ridge итерация 200, Loss: 10899769.2174
Ridge итерация 300, Loss: 10899768.8178
Ridge итерация 400, Loss: 10899768.8173
Ridge итерация 500, Loss: 10899768.8173
Ridge градиентный спуск сошелся на итерации 518
Численное решение Ridge (веса):
[9058.23223918 3219.11873685  231.5055484   586.89195344 -201.35525735
 3795.82113283 -277.25475428 -539.30752804 -553.93650275]


In [84]:
print("\n===== 4. ОЦЕНКА ОБОБЩАЮЩЕЙ СПОСОБНОСТИ =====")

def predict(X, w, scaler=None, has_bias=True):
    """
    Функция для предсказания с возможностью стандартизации
    
    Параметры:
    X - матрица признаков
    w - веса модели
    scaler - объект StandardScaler (опционально)
    has_bias - есть ли bias в матрице X
    
    Возвращает:
    Предсказания модели
    """
    if scaler is not None and not has_bias:
        # Стандартизация только признаков (без bias)
        X_scaled = np.column_stack([np.ones(X.shape[0]), scaler.transform(X)])
        return X_scaled @ w
    elif scaler is not None and has_bias:
        # Стандартизация только признаков (со второго столбца)
        X_scaled = X.copy()
        X_scaled[:, 1:] = scaler.transform(X[:, 1:])
        return X_scaled @ w
    else:
        return X @ w


===== 4. ОЦЕНКА ОБОБЩАЮЩЕЙ СПОСОБНОСТИ =====


In [85]:
# Константная модель - прогноз средним значением
y_mean = np.mean(y_train)
y_pred_const = np.full(len(y_test), y_mean)
mse_const = mean_squared_error(y_test, y_pred_const)
print(f"Константная модель (прогноз средним): MSE = {mse_const:.2f}")

# Линейная регрессия (аналитическое решение)
y_pred_lr_analytic = predict(X_test, w_analytic)
mse_lr_analytic = mean_squared_error(y_test, y_pred_lr_analytic)
print(f"Линейная регрессия (аналитическое решение): MSE = {mse_lr_analytic:.2f}")

# Линейная регрессия (численное решение)
y_pred_lr_gd = predict(X_test, w_gd, scaler=scaler, has_bias=True)
mse_lr_gd = mean_squared_error(y_test, y_pred_lr_gd)
print(f"Линейная регрессия (численное решение): MSE = {mse_lr_gd:.2f}")

# Ridge регрессия (аналитическое решение)
y_pred_ridge_analytic = predict(X_test, w_ridge_analytic)
mse_ridge_analytic = mean_squared_error(y_test, y_pred_ridge_analytic)
print(f"Ridge регрессия (аналитическое решение, alpha={alpha_optimal}): MSE = {mse_ridge_analytic:.2f}")

# Ridge регрессия (численное решение)
y_pred_ridge_gd = predict(X_test, w_ridge_gd, scaler=scaler, has_bias=True)
mse_ridge_gd = mean_squared_error(y_test, y_pred_ridge_gd)
print(f"Ridge регрессия (численное решение, alpha={alpha_optimal}): MSE = {mse_ridge_gd:.2f}")

Константная модель (прогноз средним): MSE = 36372535.61
Линейная регрессия (аналитическое решение): MSE = 14783810.68
Линейная регрессия (численное решение): MSE = 14783810.67
Ridge регрессия (аналитическое решение, alpha=10.0): MSE = 14748345.11
Ridge регрессия (численное решение, alpha=10.0): MSE = 14759775.25
